In [14]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

DATASET_DIRECTORY = r"C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset"
TYPES = ["with_mask", "without_mask"]

# All the Images Array will be appended in data list, whereas, the label of images(with_mask or without_mask) will be appended in labels
data = []
labels = []

image_size = 100

print("Loading Images present our dataset!")

for dir_type in TYPES:
    path = os.path.join(DATASET_DIRECTORY,dir_type)
    print(path);
    #listdir(path) function will list down all the diretories in the path specified
    for img in os.listdir(path):
        image_path = os.path.join(path,img)
        image = cv2.imread(image_path)
        #Printing the image just to initialize the object, else it will throw an error in cvtColor
        print(image)
        try:
            #Converting the image into a greyscale image
            greyscale_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            
            #Resizing image to (100,100)
            resized_image = cv2.resize(greyscale_image,(image_size,image_size))
            
            data.append(resized_image)
            labels.append(dir_type)
        
        except Exception as e:
            print("Exception Occured! Printing Exception: ", e)

# Now, we will firstly normalize the pixels(as the images appended are greyscale, the pixel value will range from 0 to 255)        
data = np.array(data)/255.0

# Now, reshaping the picture and converting it into a 4-Dimensional Array as neural network needs a 4D Array
data = np.reshape(data,(data.shape[0], image_size, image_size, 1))

# We've got the data as numerical values, however, the labels are still alphabetical values
# Thus we will convert them into categorical variables using LabelBinarizer from sklearn library
        
binarizer = LabelBinarizer()
labels = binarizer.fit_transform(labels)
labels = to_categorical(labels)

new_labels = np.array(labels)

np.save('data',data)
np.save('labels',new_labels)
print("Data Preprocessing Completed!")

Loading Images present our dataset!
C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset\with_mask
C:\Users\user\Desktop\Face-Mask-Detection-Using-Hardware\dataset\without_mask
Data Preprocessing Completed! Images loaded and splitted into training and testing data!
